# This is the in situ and SSS collocation code. 


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs


C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\xarray\core\merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


# Define a function to read in insitu data
- Read in the Saildrone USV file either from a local disc or using OpenDAP.
- add room to write collocated data to in situ dataset


In [2]:
def read_usv(iusv):
#    filename_usv_list = ['F:/data/cruise_data/saildrone/noaa_arctic/PMEL_2015/126/pmel_2015_sd126-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/noaa_arctic/PMEL_2015/128/pmel_2015_sd128-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/noaa_arctic/PMEL_2016/126/pmel_2016_sd126-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/noaa_arctic/PMEL_2016/128/pmel_2016_sd128-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/2019_arctic/daily_files/arctic_2019_sd1033-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/2019_arctic/daily_files/arctic_2019_sd1034-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/2019_arctic/daily_files/arctic_2019_sd1035-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/2019_arctic/daily_files/arctic_2019_sd1036-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/2019_arctic/daily_files/arctic_2019_sd1037-ALL-1_min-v1.nc',
#                        'F:/data/cruise_data/saildrone/antarctic/saildrone-gen_5-antarctica_circumnavigation_2019-sd1020-20190119T040000-20190803T043000-1440_minutes-v1.1564857794963.nc']
    filename_usv_list = ['C:/Users/gentemann/Google Drive/private/tem_saildrone/pmel_2015_sd126-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/pmel_2015_sd128-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/pmel_2016_sd126-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/pmel_2016_sd128-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/arctic_2019_sd1033-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/arctic_2019_sd1034-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/arctic_2019_sd1035-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/arctic_2019_sd1036-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/arctic_2019_sd1037-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/saildrone-gen_5-antarctica_circumnavigation_2019-sd1020-20190119T040000-20190803T043000-1440_minutes-v1.1564857794963.nc']
    name_usv_list = ['arctic2015_126',
                     'arctic2015_128',
                     'arctic2016_126',
                     'arctic2016_128',
                     'arctic2019_1033',
                     'arctic2019_1034',
                     'arctic2019_1035',
                     'arctic2019_1036',
                     'arctic2019_1037',
                    'antarctic2019']

    filename_usv = filename_usv_list[iusv]
#    if iusv==3:
#    elif iusv<3:
#    elif (iusv>3) & (iusv<8):
    ds_usv = xr.open_dataset(filename_usv)
    ds_usv.close()
#NEED TO FIND OUT IF wind_speed is to/from wind_direction ?
    if (iusv==0 or iusv==1):  #1033
        ds_usv = ds_usv.rename({'temp_air_mean':'TEMP_AIR_MEAN','rh_mean':'RH_MEAN','baro_pres_mean':'BARO_PRES_MEAN',
                                'sal_mean':'SAL_MEAN','temp_ctd_mean':'TEMP_CTD_MEAN','temp_o2_mean':'TEMP_O2_MEAN',
                                'chlor_mean':'CHLOR_MEAN','gust_wnd_mean':'GUST_WND_MEAN'})
        tem_att=ds_usv.wind_speed_mean.attrs
        ds_usv['wind_speed_mean']=ds_usv.wind_speed_mean*.51444
        ds_usv.wind_speed_mean.attrs=tem_att
        ds_usv.wind_speed_mean.attrs['units']='m s-1'
        uwnd = ds_usv.wind_speed_mean*np.cos(np.deg2rad(ds_usv.wind_direction_mean))
        vwnd = ds_usv.wind_speed_mean*np.sin(np.deg2rad(ds_usv.wind_direction_mean))
        ds_usv['UWND_MEAN']=uwnd
        ds_usv.UWND_MEAN.attrs={'standard_name':'eastward_wind','long_name':'Eastward wind speed','units':'m s-1','installed_height':'5.2'}
        ds_usv['VWND_MEAN']=vwnd
        ds_usv.VWND_MEAN.attrs={'standard_name':'northward_wind','long_name':'Northward wind speed','units':'m s-1','installed_height':'5.2'}
        ilen = ds_usv.time.shape[0]
        ds_usv['WWND_MEAN']=xr.DataArray(np.ones(ilen)*np.nan,coords={'time':ds_usv.time},dims=('time'))
        ds_usv.WWND_MEAN.attrs={'standard_name':'upward_wind_velocity','long_name':'upward wind speed','units':'m s-1','installed_height':'5.2'}
    if (iusv==2 or iusv==3):  #1033
        ds_usv = ds_usv.rename({'temp_air_mean':'TEMP_AIR_MEAN','rh_mean':'RH_MEAN','baro_pres_mean':'BARO_PRES_MEAN',
                                'sal_mean':'SAL_MEAN','temp_ctd_mean':'TEMP_CTD_MEAN','temp_o2_mean':'TEMP_O2_MEAN',
                                'chlor_mean':'CHLOR_MEAN','gust_wnd_mean':'GUST_WND_MEAN'})
        tem_att=ds_usv.wind_speed.attrs
        ds_usv['wind_speed']=ds_usv.wind_speed*.51444
        ds_usv.wind_speed.attrs=tem_att
        ds_usv.wind_speed.attrs['units']='m s-1'
        uwnd = ds_usv.wind_speed*np.cos(np.deg2rad(ds_usv.wind_direction))
        vwnd = ds_usv.wind_speed*np.sin(np.deg2rad(ds_usv.wind_direction))
        ds_usv['UWND_MEAN']=uwnd
        ds_usv.UWND_MEAN.attrs={'standard_name':'eastward_wind','long_name':'Eastward wind speed','units':'m s-1','installed_height':'5.2'}
        ds_usv['VWND_MEAN']=vwnd
        ds_usv.VWND_MEAN.attrs={'standard_name':'northward_wind','long_name':'Northward wind speed','units':'m s-1','installed_height':'5.2'}
        ilen = ds_usv.time.shape[0]
        ds_usv['WWND_MEAN']=xr.DataArray(np.ones(ilen)*np.nan,coords={'time':ds_usv.time},dims=('time'))
        ds_usv.WWND_MEAN.attrs={'standard_name':'upward_wind_velocity','long_name':'upward wind speed','units':'m s-1','installed_height':'5.2'}
    if iusv==4:  #1033
        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN','SAL_RBR_MEAN':'SAL_MEAN','CHLOR_WETLABS_MEAN':'CHLOR_MEAN'})
    if iusv==5:  #1034
        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN','SAL_RBR_MEAN':'SAL_MEAN','CHLOR_WETLABS_MEAN':'CHLOR_MEAN'})
    if iusv==6:  #1035
        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN','SAL_RBR_MEAN':'SAL_MEAN','CHLOR_WETLABS_MEAN':'CHLOR_MEAN'})
    if iusv==7:  #1036
        ds_usv = ds_usv.isel(time=slice(100,-1))                                                                   #        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN','SAL_RBR_MEAN':'SAL_MEAN','CHLOR_WETLABS_MEAN':'CHLOR_MEAN'})
        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN','SAL_RBR_MEAN':'SAL_MEAN','CHLOR_WETLABS_MEAN':'CHLOR_MEAN'})
    if iusv==8:  #1037
        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN'})
    if iusv==9:  #1037
        ds_usv = ds_usv.isel(trajectory=0).swap_dims({'obs':'time'}).rename({'latitude':'lat','longitude':'lon','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN'})#TEMP_CTD_RBR_MEAN':'TEMP_
    if (iusv==9 or iusv<=3):
        ilen = ds_usv.time.shape[0]
        ds_usv['WIND_HEIGHT_MEAN']=xr.DataArray(np.ones(ilen)*np.nan,coords={'time':ds_usv.time},dims=('time'))
        ds_usv.WIND_HEIGHT_MEAN.attrs={'long_name':'Wind measurement height','units':'m','installed_height':'5.2'}
        ds_usv['WAVE_DOMINANT_PERIOD']=xr.DataArray(np.ones(ilen)*np.nan,coords={'time':ds_usv.time},dims=('time'))
        ds_usv.WAVE_DOMINANT_PERIOD.attrs={'standard_name':'sea_surface_wave_period_at_variance_spectral_density_maximum','long_name':'Dominant wave period','units':'s','installed_height':'0.34'}
        ds_usv['WAVE_SIGNIFICANT_HEIGHT']=xr.DataArray(np.ones(ilen)*np.nan,coords={'time':ds_usv.time},dims=('time'))
        ds_usv.WAVE_SIGNIFICANT_HEIGHT.attrs={'standard_name':'sea_surface_wave_significant_height','long_name':'Significant wave height','units':'m','installed_height':'0.34'}

    return ds_usv,name_usv_list[iusv]

In [3]:
ds_usv,name=read_usv(9)
ds_usv


<xarray.Dataset>
Dimensions:                        (time: 196)
Coordinates:
    trajectory                     float32 1020.0
  * time                           (time) datetime64[ns] 2019-01-20 ... 2019-08-03
    lat                            (time) float64 ...
    lon                            (time) float64 ...
Data variables:
    SOG                            (time) float64 ...
    COG                            (time) float64 ...
    HDG                            (time) float64 ...
    HDG_WING                       (time) float64 ...
    ROLL                           (time) float64 ...
    PITCH                          (time) float64 ...
    WING_ANGLE                     (time) float64 ...
    BARO_PRES_MEAN                 (time) float64 ...
    BARO_PRES_STDDEV               (time) float64 ...
    TEMP_AIR_MEAN                  (time) float64 ...
    TEMP_AIR_STDDEV                (time) float64 ...
    RH_MEAN                        (time) float64 ...
    RH_STDDEV     


## explore the in situ data and quickly plot using cartopy
    

In [ ]:
#for NON polar data
for iusv in range(0,9):
    ds_usv,usvname = read_usv(iusv)
    print(usvname)
    #plot cruise SSS with coastlines
    if iusv<9:
        ax = plt.axes(projection=ccrs.PlateCarree())
    if iusv==9:
        ax = plt.axes(projection=ccrs.SouthPolarStereo())
    #ds_usv = ds_usv.where(np.isfinite(ds_usv.lon))
    cs1 = ax.scatter(ds_usv.lon, ds_usv.lat, s=3.0, c=ds_usv.TEMP_CTD_MEAN, edgecolor='none', cmap='jet',vmin=0,vmax=12)
    ax.coastlines(resolution='50m')
    x1,x2,y1,y2 = ds_usv.lon.min().data-2,ds_usv.lon.max().data+2,ds_usv.lat.min().data-2,ds_usv.lat.max().data+2
    ax.set_xlim(x1,x2)
    ax.set_ylim(y1,y2)
    ax.set_xticks(np.arange(x1,x2,4))
    ax.set_yticks(np.arange(y1,y2,5))
    cax = plt.colorbar(cs1)
    cax.set_label('SST (K)')
    fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/mmdb/'+usvname+'_location.png'
    plt.savefig(fig_fname, transparent=False, format='png')
    plt.clf()

In [4]:
#for polar data
for iusv in range(9,10):
    ds_usv,usvname = read_usv(iusv)
    print(usvname)
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    cs1 = ax.scatter(ds_usv.lon, ds_usv.lat, transform=ccrs.PlateCarree(),s=10.0, c=ds_usv.TEMP_CTD_MEAN, edgecolor='none', cmap='jet',vmin=0,vmax=12)
    ax.set_extent([-180, 180, -90, -45], crs=ccrs.PlateCarree())
    ax.coastlines(resolution='50m')
    cax = plt.colorbar(cs1)
    cax.set_label('SST (K)')
    fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/mmdb/'+usvname+'_location.png'
    plt.savefig(fig_fname, transparent=False, format='png')
    plt.clf()


antarctic2019


<Figure size 432x288 with 0 Axes>

In [5]:
#plot salinity
for iusv in range(9,10):
    ds_usv,usvname = read_usv(iusv)
    print(usvname)
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    cs1 = ax.scatter(ds_usv.lon, ds_usv.lat, transform=ccrs.PlateCarree(),s=10.0, c=ds_usv.SAL_MEAN, edgecolor='none', cmap='jet',vmin=33.6,vmax=34.4)
    ax.set_extent([-180, 180, -90, -45], crs=ccrs.PlateCarree())
    ax.coastlines(resolution='50m')
    cax = plt.colorbar(cs1)
    cax.set_label('SSS (psu)')
    fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/mmdb/'+usvname+'_sal_location.png'
    plt.savefig(fig_fname, transparent=False, format='png')
    plt.clf()


antarctic2019


<Figure size 432x288 with 0 Axes>